In [ ]:
%matplotlib inline
%autosave 60

import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import random
import sys
sys.path.append('../boresightUtils/')

from astropy.io import fits
from astropy.time import Time
from scipy.stats import sigmaclip
from astropy import units as u
from scipy.ndimage import gaussian_filter, median_filter
from boresightUtils import *

Autosaving every 60 seconds


In [ ]:
# imageDir = '/home/adriansh/Pictures/work/lsst/startracker/fitsFiles' # Adrian local
imageDir = '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/' # Adrian summit
cam = 'GC102'
date = '20221122'
stems = ''
globStr = os.path.join(imageDir, f'{cam}*{date}*{stems}*.fits')
files = sorted(glob.glob(globStr))
print(files)

['/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000003.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000004.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000005.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000006.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000030.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000031.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000032.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000033.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000034.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000035.fits', '/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000036.fits'

In [ ]:
darkStem = '000042'
darkStr = os.path.join(imageDir, f'{cam}*{date}*{darkStem}.fits')
darkList = sorted(glob.glob(darkStr))
print(darkList)

with fits.open(darkList[0]) as hdu:
    superDark = hdu[1].data

for darkIm in darkList[1:]:
    with fits.open(darkIm) as hdu:
        superDark += hdu[1].data

superDark = superDark / len(darkList)

['/home/adriansh95/DATA/sitcom/starTracker/tmaTests/procFits/GC102_O_20221122_000042.fits']


In [ ]:
driftNum0 = '000062'
driftNum1 = '000063'
arcNum = '000044'
driftIm0 = f'{cam}_O_{date}_{driftNum0}.fits'
driftIm1 = f'{cam}_O_{date}_{driftNum1}.fits'
arcIm = f'{cam}_O_{date}_{arcNum}.fits'

with fits.open(os.path.join(imageDir, driftIm0)) as hdu:
    dIm0 = hdu[1].data
    t0 = Time(hdu[0].header['DATE-BEG'])
       
with fits.open(os.path.join(imageDir, driftIm1)) as hdu:
    dIm1 = hdu[1].data
    t1 = Time(hdu[0].header['DATE-BEG'])
    phi = hdu[0].header['AZEND']

with fits.open(os.path.join(imageDir, arcIm)) as hdu:
    arcImage = hdu[1].data
    azStart = hdu[0].header['AZSTART']
    azEnd = hdu[0].header['AZEND']
    arcT0 = Time(hdu[0].header['DATE-BEG'])
    arcT1 = Time(hdu[0].header['DATE-END'])
   
thetaInit = azStart - phi
dt = t1 - t0
dt = dt.to(u.s).value
totalDrift = measure_drift(dIm0, dIm1)
driftRateRC = totalDrift / dt
driftRate = np.array([[driftRateRC[1]], [-driftRateRC[0]]])

dTheta = azEnd - azStart
arcDt = arcT1 - arcT0
arcDt = arcDt.to(u.s).value
slewRate = dTheta / arcDt
print(driftRate, slewRate)

[[-6.91927512]
 [-5.71114772]] -7.01377931695846


In [ ]:
rng = np.random.default_rng()
height, width = arcImage.shape
xMu = width / 2
yMu = height / 2

configDict = {
    'expTime': arcDt,
    'slewRate': slewRate,
    'driftVel': driftRate,
    'threshold': 0.3,
    'rng': rng,
    'imageWidth': width,
    'imageHeight': height,
    'gaussianBlurSigma': 1,
    'xMu': xMu,
    'yMu': yMu,
    'pointingSigma': 100,
    'nStars': 25,
    }

biases = calculate_drift_bias(configDict, thetaInit=thetaInit, tol=0.001, nTrials=20)
bias = biases.mean(axis=0)

In [ ]:
print(biases, bias, biases.std(axis=0), biases.std(axis=0)/np.sqrt(len(biases)))

In [30]:
arcImage = arcImage - superDark
gSig = 7
nStd = 4
arcImage = gaussian_filter(arcImage, sigma=gSig)
c, low, high = sigmaclip(arcImage)
mean = c.mean()
std = c.std()
tArcImage = np.where(arcImage > mean+nStd*std, 1, 0)
nTrials = 20
boresights = np.zeros((nTrials, 2))

for i in range(nTrials):
    boresights[i] = find_boresight_naive(tArcImage)

In [31]:
print(boresights, boresights.std(axis=0), boresights.std(axis=0)/np.sqrt(len(boresights)))

[[ 920.55580143 1815.21904845]
 [ 920.55954887 1814.51687979]
 [ 921.10778749 1815.14284205]
 [ 921.41636264 1814.90135137]
 [ 920.37217954 1815.79380482]
 [ 921.58160123 1814.48564638]
 [ 921.35700227 1814.27725601]
 [ 919.760524   1815.76741575]
 [ 920.4466606  1815.51839988]
 [ 920.26292944 1814.90556337]
 [ 920.32261306 1815.24928941]
 [ 920.77684454 1815.20394826]
 [ 919.44491726 1815.07117534]
 [ 920.70575051 1815.36232161]
 [ 920.97459391 1815.44380815]
 [ 921.55900533 1813.91456464]
 [ 921.10368182 1815.1499765 ]
 [ 920.62099775 1815.07467003]
 [ 919.57356458 1815.79056202]
 [ 919.65570345 1814.56043887]] [0.63521698 0.49827981] [0.14203883 0.11141875]


In [4]:
boresight = boresights.mean(axis=0) - bias
boresightSigma = boresights.std(axis=0)/np.sqrt(len(boresights)) + biases.std(axis=0)/np.sqrt(len(biases))
plot_arc_image(tArcImage, boresight=boresight, saveFig=True, figName='GC102_O_20221122_000043_marked.png')
print(boresight, boresightSigma)

NameError: name 'boresights' is not defined